In [1]:
import os
os.chdir('../..')

In [2]:
from os.path import join

import numpy as np
import pandas as pd

import analysis.stats as st

dataframe_dir = join('results', 'dataframes')

# Load all results
df_total_k = pd.read_pickle(join(dataframe_dir, 'df_total_k.pkl'))
df_total_z = pd.read_pickle(join(dataframe_dir, 'df_total_z.pkl'))

gait_params = df_total_k.select_dtypes(float).columns

df_trials_k = df_total_k.groupby('trial_id').median()
df_trials_z = df_total_z.groupby('trial_id').median()

df_sides_k = df_total_k.groupby(['trial_id', 'side']).median()
df_sides_z = df_total_z.groupby(['trial_id', 'side']).median()

sides = df_sides_k.index.get_level_values(level=1).unique()

# bland_alt_tuples = []

df_icc = pd.DataFrame(index=gait_params, 
                      columns=pd.MultiIndex.from_product([['ICC(2, 1)', 'ICC(3, 1)'], sides]))

for param in gait_params:
    
    for side in sides:

        measures_1 = df_sides_k[param].loc[:, side]
        measures_2 = df_sides_z[param].loc[:, side]

#     differences = st.relative_difference(measures_1, measures_2)
#     bland_alt = st.bland_altman(differences)
#     bland_alt_tuples.append(bland_alt)

        measures = np.column_stack((measures_1, measures_2))
        
        icc_21 = st.icc(measures, type=(2, 1))
        icc_31 = st.icc(measures, type=(3, 1))
        
        df_icc.loc[param].loc['ICC(2, 1)', side] = icc_21
        df_icc.loc[param].loc['ICC(3, 1)', side] = icc_31
    
# df_bland = pd.DataFrame.from_records(
#     bland_alt_tuples, index=gait_params, columns=bland_alt._fields)


In [3]:
df_icc

ICC(2, 1)            ICC(3, 1)          
                              L          R         L         R
absolute_step_length   0.899588   0.980283  0.960733  0.980845
stance_percentage       0.84068   0.709956  0.840877  0.731394
step_length            0.915143   0.977665  0.962555  0.981453
stride                0.0416221  0.0347518  0.429939  0.340159
stride_length          0.977558   0.985274  0.977339  0.986856
stride_time            0.695287   0.699823  0.928107  0.904932
stride_velocity        0.783511   0.800528  0.917985   0.93491
stride_width           0.608582     0.5911  0.767156  0.800121